In [1]:
# Import packages

import pandas as pd
import numpy as np
import altair as alt

In [2]:
# Tools

import functools
def conjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)

In [3]:
%%html
    <style>
        @import url('https://fonts.googleapis.com/css?family=Encode+Sans:400');
        @import url('https://fonts.googleapis.com/css?family=Open+Sans:400');
        @import url('https://fonts.googleapis.com/css?family=Open+Sans:700');
    </style>

In [4]:
def catalyst(*args, **kwargs):
    font = "Open Sans"
    labelFont = "Open Sans" 
    sourceFont = "Open Sans"
    # Axes
    axisColor = "#000000"
    gridColor = "#DEDDDD"
    # Colors
    main_palette = ["#2980B9", 
              "#003366",
              "#FFAE19",
              "#FF4500",
              "#438B28",
              "#45B39D",
              "#99ADC1",
              "#212F3C"]
    return {
        "width": 800, 
        "height": 400, 
        "config": {
            "title": {
                "fontSize": 18,
                "font": font,
                "anchor": "center",
                "fontColor": "#000000",
            },
            "axisX": {
                "domain": True,
                "domainColor": axisColor,
                #"domainWidth": 1,
                "grid": False,
                "labelFont": labelFont,
                "labelFontSize": 14,
                #"labelAngle": 0,
                #"tickColor": axisColor,
                #"tickSize": 5, 
                "titleFont": font,
                "titleFontSize": 13,
                "titleFontWeight": "normal",
                #"titlePadding": 10, 
                "title": "X Axis Title (units)", 
                #"titleAngle":0
            },
            "axisY": {
                "domain": True,
                "grid": False,
                "gridColor": gridColor,
                "gridWidth": 1,
                "labelFont": labelFont,
                "labelFontSize": 14,
                "labelAngle": 0,
                "ticks": False, 
                "titleFont": font,
                "titleFontSize": 14,
                "titleFontWeight": "normal",
                "titlePadding": 10, 
                "title": "Y Axis Title (units)",
                #"titleY": -10, 
                #"titleX": 18,
                #"titleAngle": 0 #-90
            },
            "range": {
                "category": main_palette
            },
            "legend": {
                "labelFont": labelFont,
                "labelFontSize": 14,
                "symbolType": "square", 
                "symbolSize": 100, 
                "titleFont": font,
                "titleFontSize": 14,
                "titleFontWeight": "normal",
                "title": "Legend", 
                "orient": "right", 
                "offset": 0, 
            },
            "tooltip":{
                "font": "Open Sans"
            },
            "font":{
                "font": "Open Sans"
            },
            "view": {
                "stroke": "transparent", 
        ### MARKS CONFIGURATIONS ###
           "text": {
               "font": "Open Sans",
               "color": main_palette,
               "fontSize": 14,
               "align": "right",
               "fontWeight": 400,
               "size": 14,
           }, 
           "bar": {
                "size": 40,
                "binSpacing": 1,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": main_palette,
                "stroke": False,
           },
            }
        }
    }

In [5]:
alt.themes.register('catalyst', catalyst)
alt.themes.enable('catalyst');

In [6]:
# Load Nigeria MTF Data: Finance Details
#Total number of households surveyed in the overall survey: 3,668

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("../../data/nigeria_finance.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
finance = pd.concat(mtf_list, axis=0)
finance = pd.DataFrame(data = finance)

In [7]:
# Load Nigeria MTF Data: Cooking Details

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("../../data/nigeria_cooking.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
cooking = pd.concat(mtf_list, axis=0)
cooking = pd.DataFrame(data = cooking)

In [8]:
# Load Nigeria MTF Data: Electricity Details

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("../../data/nigeria_grid_access.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
electricity = pd.concat(mtf_list, axis=0)
electricity = pd.DataFrame(data = electricity)
electricity['i19a'] = finance['i19a']

In [9]:
# Load Nigeria MTF Data: Electricity-2 Details

chunksize = 10

mtf_list = []
chunksize = 10
for chunk in pd.read_csv("../../data/nigeria_grid_access_ext.csv", encoding='latin-1', chunksize = chunksize):
    mtf_list.append(chunk)
    
electricity2 = pd.concat(mtf_list, axis=0)
electricity2 = pd.DataFrame(data = electricity2)
electricity2['i19a'] = finance['i19a']
electricity2['c2'] = electricity['c2']

In [10]:
#find the primary stove used by each household
primary_stove = []
for i in range(cooking.shape[0]):
    if cooking['i35'][i] == 'Yes':
        primary_stove.append(cooking['i3'][i])
    else:
        primary_stove.append('NaN')
finance['stove'] = primary_stove

In [11]:
cooking_short = cooking[['Unnamed: 0','hh_id', 'i3','i35']].rename(columns={'hh_id': 'hh_id_right'})
electricity = electricity.join(cooking_short.set_index('Unnamed: 0'), on = 'Unnamed: 0', how = 'inner')

In [12]:
#Find primary stove for households in electricity dataset
#(length of initial primary_stove list different from electricity dataframe)
primary_stove = []
for i in range(electricity.shape[0]):
    if electricity['i35'][i] == 'Yes':
        primary_stove.append(electricity['i3'][i])
    else:
        primary_stove.append('NaN')
electricity['stove'] = primary_stove
electricity2['stove'] = primary_stove

In [ ]:
# Have a bank account

# Data source
alt.data_transformers.disable_max_rows()
source = finance[finance[['b16']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['b16'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'c2', 'i19a', 'b16', 'stove']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('b16', # Change variable
              sort=alt.EncodingSortField(field='b16', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
 #   color = 'b16:N', # Change variable
    tooltip = 'b16' # Change variable
).properties(
    height = 400,
    width = 500
)

# Locality Filter
locality_options = [None, 0, 1]
locality_labels = ['National', 'Urban', 'Rural']

locality_dropdown=alt.binding_select(
     options= locality_options, 
     labels = locality_labels, 
     name = "Locality Breakdown  ")
locality_select=alt.selection_single(
     fields=['locality'],
     bind=locality_dropdown)

# Tier Filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, 
                                 labels = tier_labels, 
                                 name = "Tiers  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# National Grid Connection Filter
connection = [None, 'Yes', 'No']
connection_labels = ['All Responses', 'Yes', 'No']
connection_dropdown=alt.binding_select(options=connection, 
                                 labels = connection_labels, 
                                 name = "Is the household connected to the national grid?  ")
connection_select=alt.selection_single(fields=['c2'],
                                bind=connection_dropdown)

# Primary Cooking Fuel Filter
fuel = [None, 'Wood purchased', 'Wood collected', 'Charcoal', "Kerosene", 'Crop Residue/Plant Biomass',
             'LPG/cooking Gas', 'Garbage', 'Electric', 'Piped Natural Gas', 'Charcoal Briquettes', 'Coal Briquette',
              'Biomass Briquette', 'Coal/lignite', 'Other, specify']
fuel_labels = ['All Responses', 'Wood Purchased', 'Wood Collected', 'Charcoal', 'Kerosene',
                     'Crop Residue/Plant Biomass', 'LPG/Cooking Gas', 'Garbage', 'Electricity', 'Piped Natural Gas',
                    'Charcoal Briquettes', 'Coal Briquette', 'Biomass Briquette', 'Coal/Lignite', 'Other']
fuel_dropdown=alt.binding_select(options= fuel, 
                                 labels = fuel_labels, 
                                 name = "Primary source of cooking fuel  ")
fuel_select=alt.selection_single(fields=['i19a'],
                                bind=fuel_dropdown)

# Cookstove Connection Filter
stove = [None, "3-Stone/Open Fire Stove", "Manufactured Stove Traditional", "Koresone Stove", "LPG/Natural Gas Stove",
        "Electric Stove"]
stove_labels = ['All Responses', '3-Stone/Open Fire Stove', 'Manufactured Stove Traditional', "Koresone Stove",
               "LPG/Natural Gas Stove", "Electric Stove"]
stove_dropdown=alt.binding_select(options=stove, 
                                 labels = stove_labels, 
                                 name = "Primary cookstove  ")
stove_select=alt.selection_single(fields=['stove'],
                                bind=stove_dropdown)

# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["Does anyone in the household have a bank account at a formal institution?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Where is the bank account located

# Data source
alt.data_transformers.disable_max_rows()
source = finance[finance[['account_institution']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['account_institution'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'account_institution', 'i19a', 'c2', 'stove']] # Change variables here
,
# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('account_institution', # Change variable
              sort=alt.EncodingSortField(field='account_institution', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
 #   color = 'account_institution:N', # Change variable
    tooltip = 'account_institution' # Change variable
).properties(
    height = 400,
    width = 500
)

# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
 ).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["At which institution is this account or savings?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Does the household have an informal savings account

# Data source
alt.data_transformers.disable_max_rows()
source = finance[finance[['b18']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['b18'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'b18', 'i19a', 'c2', 'stove']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('b18', # Change variable
              sort=alt.EncodingSortField(field='b18', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
#    color = 'b18:N', # Change variable
    tooltip = 'b18' # Change variable
).properties(
    height = 400,
    width = 500
)

# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
 ).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["Does anyone in the household use an informal savings groups (adashi/esusu/ajo) to save money?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Where is the informal bank account located

# Data source
alt.data_transformers.disable_max_rows()
source = finance[finance[['informal_institution']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['informal_institution'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'informal_institution', 'i19a', 'c2', 'stove']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('informal_institution', # Change variable
              sort=alt.EncodingSortField(field='informal_institution', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
#    color = 'informal_institution:N', # Change variable
    tooltip = 'informal_institution' # Change variable
).properties(
    height = 400,
    width = 500
)

# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
 ).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["What type of informal savings group do members of this household use?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Save as an HTML file
#final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Number of mobile money accounts

# Data source
alt.data_transformers.disable_max_rows()
source = finance[finance[['b21']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['b21'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'b21', 'i19a', 'c2', 'stove']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('b21', # Change variable
              sort=alt.EncodingSortField(field='b21', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
#    color = 'b21:N', # Change variable
    tooltip = 'b21' # Change variable
).properties(
    height = 400,
    width = 500
)


# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
 ).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["Do you have a mobile money account?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Number of respondents that use mobile money accounts to make payments

# Data source
alt.data_transformers.disable_max_rows()
source = finance[finance[['b22']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['b22'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'b22', 'i19a', 'c2', 'stove']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('b22', # Change variable
              sort=alt.EncodingSortField(field='b22', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
 #   color = 'b22:N', # Change variable
    tooltip = 'b22' # Change variable
).properties(
    height = 400,
    width = 500
)

# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
 ).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["Do you use mobile money to make payments over the mobile phone?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Have you used the mobile money account within the past 90 days?

# Data source
alt.data_transformers.disable_max_rows()
source = finance[finance[['b23']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['b23'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'b23', 'i19a', 'c2', 'stove']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('b23', # Change variable
              sort=alt.EncodingSortField(field='b23', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
#    color = 'b23:N', # Change variable
    tooltip = 'b23' # Change variable
).properties(
    height = 400,
    width = 500
)



# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
 ).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["Have you used your mobile money account in the past 90 days?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# Most common method used to make electricity payment

# Data source
alt.data_transformers.disable_max_rows()
source = electricity[electricity[['c58']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['c58'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality', 'c58', 'i19a', 'c2', 'stove']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('c58', # Change variable
              sort=alt.EncodingSortField(field='c58', # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
 #   color = 'c58:N', # Change variable
    tooltip = 'c58' # Change variable
).properties(
    height = 400,
    width = 500
)

# Percentage of responses shown in the chart
pct = round(df.shape[0] / finance.shape[0] * 100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
 ).add_selection(
    stove_select
).transform_filter(
    stove_select 
).add_selection(
    fuel_select
).transform_filter(
    fuel_select 
).add_selection(
    connection_select
).transform_filter(
    connection_select 
).add_selection(
     locality_select
 ).transform_filter(
     locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["What is the most common way you make your electricity bill payment?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [ ]:
# How many mobile phones does your household have?

# Data Source
alt.data_transformers.disable_max_rows()
source = electricity2[electricity2[['c183']].notnull().all(1)]

col1 = source['c183'] # != 'NaN'
col2 = source['elc_aggr_tier'] != 'NaN'

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier','c183', 'locality','i19a', 'c2', 'stove']] #change variable here

# Percentage of responses shown in the chart
pct = round(df.shape[0] / electricity2.shape[0] * 100, 2) #change dataframe variable name

# Draw the Chart
final = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'
).mark_bar(color = '#304ca2'
).encode(
    alt.X("c183:Q", #change variable
          bin=alt.Bin(extent=[1, 15], step=1), # Change the step to adjust bin size
          scale = alt.Scale(domain=(1,15),clamp = True), # Change the axes limits
         axis = alt.Axis(title = "Response")),
    alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = "Percent"),
    tooltip = [alt.Tooltip('c183:Q', title = "Response (Binned)"), #Change variable
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=20) # Specify the maximum number of bins
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    height = 400,
    width = 500,
    title={
      "text": ["How many mobile phones do the household members own combined?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

#### Loan Source and Mobile Money Account Activity Charts

In [13]:
#Prep Loan Source Variable Columns to visualize as a single bar chart
finance = finance.rename(columns={'b20__1': "Commercial/government bank", "b20__2": "Cooperative credit union/SACCO",
                                  "b20__3": "Microfinance institution", "b20__4": "Rural bank",'b20__5': "State loan",
                                  'b20__6': "NGO", 'b20__7':"Business firm", 'b20__8':"Employer",
                                  'b20__9': "Moneylender/Shylock", 'b20__10': "Shop",
                                  'b20__11': "Relative/friend/neighbor", 'b20__12': "Mobile money services",
                                  'b20__13': "Cannot get a loan or credit", 'b20__555': "Other"})

loan_wide = finance[['hh_id',"Commercial/government bank", "Cooperative credit union/SACCO", "Microfinance institution",
                     "Rural bank", "State loan", "NGO", "Business firm", "Employer","Moneylender/Shylock",
                     "Shop", "Relative/friend/neighbor", "Mobile money services", "Cannot get a loan or credit",
                     "Other"]].melt('hh_id', var_name='Source', value_name='Response')
loan_wide

,hh_id,Source,Response
0,1000506120305,Commercial/government bank,0
1,1000506120401,Commercial/government bank,1
2,1000506120601,Commercial/government bank,0
3,1000506120602,Commercial/government bank,0
4,1000506120901,Commercial/government bank,0
...,...,...,...
57941,12019518882801,Other,0
57942,12019518882901,Other,0
57943,12019518883001,Other,0
57944,12019518883102,Other,0


In [14]:
#Check number of responses per category
loan_wide.groupby(['Source'], as_index=False).sum()

,Source,hh_id,Response
0,Business firm,29236904485113040,25
1,Cannot get a loan or credit,29236904485113040,952
2,Commercial/government bank,29236904485113040,377
3,Cooperative credit union/SACCO,29236904485113040,37
4,Employer,29236904485113040,19
5,Microfinance institution,29236904485113040,57
6,Mobile money services,29236904485113040,7
7,Moneylender/Shylock,29236904485113040,8
8,NGO,29236904485113040,23
9,Other,29236904485113040,40


In [15]:
# 2 steps: Calculate number of responses per category and total number of responses for this question
# (this is distinct from number of survey respondents)
loan_group = loan_wide.groupby(['Source']).agg({'Response': 'sum'})
loan = loan_wide.groupby(['Source']).agg({'Response': 'count'})
loan_percent = loan_group.div(loan, level='Source')
loan_percent = loan_percent.reset_index().sort_values(by='Response', ascending=False)
loan_percent

,Source,Response
10,Relative/friend/neighbor,0.670693
1,Cannot get a loan or credit,0.230007
2,Commercial/government bank,0.091085
12,Shop,0.033100
5,Microfinance institution,0.013771
9,Other,0.009664
3,Cooperative credit union/SACCO,0.008939
0,Business firm,0.006040
8,NGO,0.005557
4,Employer,0.004590


In [23]:
chart = alt.Chart(loan_percent).encode(
    x=alt.X('Source:N', sort=alt.EncodingSortField(field='Source', # Change variable 
                                         op="count", 
                                         order='descending')),
    y=alt.Y('Response:Q', axis = alt.Axis(title = '', format = '%')),
  #  color = alt.Color('Source:N'), # Change variable
    tooltip = 'Source' # Change variable
).properties(
    height = 400,
    width = 500
)

# Percentage of responses shown in the chart
pct = round(len(pd.unique(loan_wide[loan_wide[['Response']].notnull().all(1)]['hh_id'])) /
            3668 *100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('Response:Q', 
                    format = '.2%')
).properties(
    title={
      "text": ["If you can get a loan/credit, what are the sources of credit/loans?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
))

final

alt.LayerChart(...)

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

In [24]:
#Prep Mobile Money Account Activity Columns to visualize as a single bar chart
finance = finance.rename(columns={'b24__1': "Receive money from family/friends/other",
                                  "b24__2": "Transfer credit to family/relatives",
                                  "b24__3": "Top up credit", "b24__4": "Receive NGO/State support",
                                  'b24__5': "Pay for electricity", 'b24__6': "Pay for water",
                                  'b24__7': "Internet top-up/credit", 'b24__8':"Commercial Purchases",
                                  'b24__9': "Insurance", 'b24__10': "Loan payments",
                                  'b24__11': "Savings", 'b24__12': "Get small loans from mobile provider",
                                  'b24__555': "Other"})

mobile_money_activity_wide = finance[['hh_id',"Receive money from family/friends/other",
                                      "Transfer credit to family/relatives", "Top up credit", "Receive NGO/State support",
                                      "Pay for electricity", "Pay for water","Internet top-up/credit",
                                      "Commercial Purchases","Insurance", "Loan payments","Savings",
                                      "Get small loans from mobile provider","Other"]].melt(
    'hh_id', var_name='Source', value_name='Response')
mobile_money_activity_wide

,hh_id,Source,Response
0,1000506120305,Receive money from family/friends/other,NaN
1,1000506120401,Receive money from family/friends/other,NaN
2,1000506120601,Receive money from family/friends/other,NaN
3,1000506120602,Receive money from family/friends/other,NaN
4,1000506120901,Receive money from family/friends/other,NaN
...,...,...,...
57941,12019518882801,Other,NaN
57942,12019518882901,Other,NaN
57943,12019518883001,Other,NaN
57944,12019518883102,Other,NaN


In [25]:
#Check number of responses per category
mobile_money_activity_wide.groupby(['Source'], as_index=False).sum()

,Source,hh_id,Response
0,Commercial Purchases,29236904485113040,3.0
1,Get small loans from mobile provider,29236904485113040,2.0
2,Insurance,29236904485113040,0.0
3,Internet top-up/credit,29236904485113040,3.0
4,Loan payments,29236904485113040,1.0
5,Other,58473808970226080,40.0
6,Pay for electricity,29236904485113040,0.0
7,Pay for water,29236904485113040,0.0
8,Receive NGO/State support,29236904485113040,0.0
9,Receive money from family/friends/other,29236904485113040,39.0


In [26]:
# 2 steps: Calculate number of responses per category and total number of responses for this question
# (this is distinct from number of survey respondents)
mobile_money_activity_group = mobile_money_activity_wide.groupby(['Source']).agg({'Response': 'sum'})
mobile_money_activity = mobile_money_activity_wide.groupby(['Source']).agg({'Response': 'count'})
mobile_money_activity_percent = mobile_money_activity_group.div(mobile_money_activity, level='Source')
mobile_money_activity_percent = mobile_money_activity_percent.reset_index().sort_values(by='Response', ascending=False)
mobile_money_activity_percent

,Source,Response
12,Transfer credit to family/relatives,0.735294
9,Receive money from family/friends/other,0.573529
11,Top up credit,0.308824
10,Savings,0.147059
0,Commercial Purchases,0.044118
3,Internet top-up/credit,0.044118
1,Get small loans from mobile provider,0.029412
4,Loan payments,0.014706
5,Other,0.009508
2,Insurance,0.000000


In [28]:
chart = alt.Chart(mobile_money_activity_percent).encode(
    x=alt.X('Source:N', sort=alt.EncodingSortField(field='Source', # Change variable 
                                         op="count", 
                                         order='descending')),
    y=alt.Y('Response:Q', axis = alt.Axis(title = '', format = '%')),
  #  color = alt.Color('Source:N'), # Change variable
    tooltip = 'Source' # Change variable
).properties(
    height = 400,
    width = 500
)

# Percentage of responses shown in the chart
pct = round(len(pd.unique(mobile_money_activity_wide[mobile_money_activity_wide[['Response']].notnull().all(1)]['hh_id'])) /
                3668 *100, 2) #change dataframe variable name

# Final Chart with bells and whistles
final = (chart.mark_bar() + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('Response:Q', 
                    format = '.2%')
).properties(
    title={
      "text": ["How do you use the mobile money services?"], 
      "subtitle": ["You are viewing {}% of responses from the Nigeria Multi-Tiered Framework Survey.".format(pct), " "],
      "color": "black",
      "subtitleColor": "black"
    }
))

final

alt.LayerChart(...)

In [ ]:
# Save as an HTML file
final.save('final.html')

# Save as a JSON file
final.save('final.json')

#### Chart Templates

In [ ]:
# Bar Chart Example
# B.14 from the Kenya MTF: What is your household’s main source of drinking water?

# Data source
alt.data_transformers.disable_max_rows()
source = module[module[['b_b_14']].notnull().all(1)] # Change your variables here

col1 = source['elc_aggr_tier'] != 'NaN'
col2 = source['b_b_14'] != 'NaN' # Change variables here

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier', 'locality_ur', 'b_b_14']] # Change variables here

# Draw the chart 
chart = alt.Chart(df).transform_joinaggregate(
    total = 'count(*)'
).transform_calculate(
    pct = '1 / datum.total'
).encode(
    x = alt.X('b_b_14', # Change variable
              sort=alt.EncodingSortField(field="b_b_14", # Change variable 
                                         op="count", 
                                         order='descending'),
              axis = alt.Axis(title = " ")),
    y = alt.Y('sum(pct):Q', axis = alt.Axis(title = '', format = '%')),
    tooltip = 'b_b_14' # Change variable
).properties(
    height = 400,
    width = 500
)

# Locality Filter
locality_options = [None, 'Urban', 'Rural']
locality_labels = ['National', 'Urban', 'Rural']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality_ur'],
    bind=locality_dropdown)

# Tier Filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, 
                                 labels = tier_labels, 
                                 name = "Tiers  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Percentage of responses shown in the chart
pct = df.shape[0] / module.shape[0]

# Final Chart with bells and whistles
final = (chart.mark_bar(color = '#304ca2') + chart.mark_text( # This blue is from the AIP site.
    align = 'center', 
    baseline='bottom'
).encode(
    text = alt.Text('sum(pct):Q', 
                    format = '.2%'))
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    title={
      "text": ["Example Bar Chart (YOUR TITLE HERE)"], 
      "subtitle": ["You are viewing XX% of responses from the Nigeria Multi-Tiered Framework Survey.", " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Histogram Example
# How many solar-powered systems do you have?

# Data Source
alt.data_transformers.disable_max_rows()
source = solar[solar[['sum_solar']].notnull().all(1)]

col1 = source['sum_solar'] # != 'NaN'
col2 = source['elc_aggr_tier'] != 'NaN'

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier','sum_solar', 'locality']]

# Draw the Chart
final = alt.Chart(df).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'
).mark_bar(color = '#304ca2'
).encode(
    alt.X("sum_solar:Q", 
          bin=alt.Bin(extent=[1, 15], step=1), # Change the step to adjust bin size
          scale = alt.Scale(domain=(1,15),clamp = True), # Change the axes limits
         axis = alt.Axis(title = "Response")),
    alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = "Percent"),
    tooltip = [alt.Tooltip('b_b_9:N', title = "Response (Binned)"), 
               alt.Tooltip('sum(pct):Q', format = '.2%', title = "Percentage")]
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=20) # Specify the maximum number of bins
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).properties(
    height = 400,
    width = 500,
    title={
      "text": ["How many solar-powered systems do you own?"], 
      "subtitle": ["You are viewing 1% of responses from the Nigeria Multi-Tiered Framework Survey.", " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Heatmap Example

# Data
alt.data_transformers.disable_max_rows()
source = cooking[cooking[['i19a', 'primary_stove']].notnull().all(1)]

col1 = source['i19a'] != 'NaN'
col2 = source['elc_aggr_tier'] != 'NaN'

source = source[conjunction(col1,col2)]
df = source[['elc_aggr_tier','i19a', 'primary_stove', 'locality']]

# Configure common options
base = alt.Chart(df).transform_aggregate(
    chart_count='count()',
    groupby=['i19a', 'primary_stove']
).encode(
    alt.X('i19a:O', 
          scale=alt.Scale(paddingInner=0), 
          title = "Primary Cooking Fuel"),
    alt.Y('primary_stove:O', 
          scale=alt.Scale(paddingInner=0), 
          title = "Primary Cooking Stove"),
)

# Configure heatmap
heatmap = base.mark_rect().encode(
    color=alt.Color('chart_count:Q',
        scale=alt.Scale(scheme='blues'),
        legend=None
    )
)

# Configure text
text = base.mark_text(baseline='middle'
).transform_joinaggregate(
    count = 'count(chart_count)',
    groupby = ['i19a', 'primary_stove']
).transform_calculate(
    pct = alt.datum.chart_count / df.shape[0]
).encode(
    text=alt.Text('pct:Q', format = '.2%'),
    color=alt.condition(
        alt.datum.chart_count < 300, # Change the color of the text here
        alt.value('black'),
        alt.value('white')
    )
)

# Locality Filter
locality_options = [None, 0, 1]
locality_labels = ['National', 'Urban', 'Rural']

locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality'],
    bind=locality_dropdown)

# Tier Filter
tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Tiers  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# Roof filter
#roofs = [None, 'Corrugated iron sheet', 'Concrete/Cement', 'Thatch', 'Wood and mud', 'Bamboo/Reed', 
 #        'Plastic canvas', 'Asbestos', 'Bricks']
#roof_labels = ['All roof types', 'Corrugated iron sheet', 'Concrete/Cement', 'Thatch', 'Wood and mud', 'Bamboo/Reed', 
 #        'Plastic canvas', 'Asbestos', 'Bricks']
#roof_dropdown=alt.binding_select(options=roofs, labels = roof_labels, name = "Roofs  ")
#roof_select=alt.selection_single(fields=['b_b_11'],
 #                               bind=roof_dropdown)

# Draw the chart
final = (heatmap + text).properties(
    height = 300,
    width = 800
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
# ).add_selection(
#     roof_select
# ).transform_filter(
#     roof_select
).properties(
    title={
      "text": ['Commonly Used Cooking Fuels and Stoves'], 
    #  "subtitle": ["You are viewing XX% of responses from the Kenya Multi-Tiered Framework Survey.", " "],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Step 1: create the scatterplot
# How much would you sell your land for?

df = module[module[['n_n_1b', 'n_n_1c', 'n_n_1b_unit']].notnull().all(1)]

# Filters
locality_options = [None, 'Urban', 'Rural']
locality_labels = ['National', 'Urban', 'Rural']
locality_dropdown=alt.binding_select(
    options= locality_options, 
    labels = locality_labels, 
    name = "Locality Breakdown  ")
locality_select=alt.selection_single(
    fields=['locality_ur'],
    bind=locality_dropdown)

tiers = [None, 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_labels = ['All Tiers', 'Tier 0', 'Tier 1', 'Tier 2', 'Tier 3', 'Tier 4', 'Tier 5']
tier_dropdown=alt.binding_select(options=tiers, labels = tier_labels, name = "Tiers  ")
tier_select=alt.selection_single(fields=['elc_aggr_tier'],
                                bind=tier_dropdown)

# You need 2 unit filters to apply to both the scatterplot and histogram.
units = ['Acre', 'sq metres','Other']
unit_labels = ['Acre', 'Square meter','Other']
unit_dropdown=alt.binding_select(options=units, labels = unit_labels, name = "Units  ")
unit_select=alt.selection_single(fields=['n_n_1b_unit:N'],
                                bind=unit_dropdown)

units = [None, 'Acre', 'sq metres','Other']
unit_labels = ['All units', 'Acre', 'Square meter','Other']
unit_dropdown=alt.binding_select(options=units, labels = unit_labels, name = "Units  ")
unit_select=alt.selection_single(fields=['n_n_1b_unit'],
                                 bind=unit_dropdown)
 
# Scatterplot    
points = alt.Chart(df).mark_point().encode(
    x=alt.X('n_n_1b:Q',
           axis = alt.Axis(title = 'What is the total size of the land?'),
           scale = alt.Scale(domain=(1,100),clamp = True)),
    y=alt.Y('n_n_1c:Q',
            axis=alt.Axis(title='How much would you sell your land for?'),
           scale = alt.Scale(domain=(1,70000000),clamp = True)),
    color = alt.Color('n_n_1b_unit:N', legend = alt.Legend(title = "Units"))
)


scatterplot = (points).properties(
    height = 300,
    width = 500
).add_selection(brush)

scatterplot

In [ ]:
# Step 2: add the histogram
df = module[module[['n_n_1b']].notnull().all(1)]

points = alt.Chart(df).mark_point().encode(
    x=alt.X('n_n_1b:Q',
           axis = alt.Axis(title = 'What is the total size of the land?'),
           scale = alt.Scale(domain=(1,100),clamp = True)),
    y=alt.Y('n_n_1c:Q',
            axis=alt.Axis(title='How much would you sell your land for?'),
           scale = alt.Scale(domain=(1,70000000),clamp = True)),
    color = alt.Color('n_n_1b_unit:N', legend = alt.Legend(title = "Units"))
)

scatterplot = (points).properties(
    height = 300,
    width = 500
).add_selection(brush)

chart = alt.Chart(df).mark_bar(
).transform_joinaggregate(
    total='count(*)'
).transform_calculate(
    pct='1 / datum.total'
).encode(
    alt.X("n_n_1b", 
          bin=alt.Bin(extent=[0, 100], step=5), 
          scale = alt.Scale(domain=(0,100),clamp = True),
         axis = alt.Axis(title = "Units of Land")),
    alt.Y('sum(pct):Q', axis=alt.Axis(format='%'), title = "Percent"),
    tooltip = alt.Tooltip('sum(pct):Q', format = '.2%')
).transform_bin(
    "mbin",
    field="m",
    bin=alt.Bin(maxbins=20)
)

# Draw the chart
histogram = chart.properties(
    height = 300,
    width = 500,
    title = ''
).transform_filter(brush)


scatterplot & histogram

In [ ]:
# Finally, add filters

final = (scatterplot & histogram
).add_selection(
    locality_select
).transform_filter(
    locality_select
).add_selection(
    tier_select
).transform_filter(
    tier_select 
).add_selection(
    unit_select
).transform_filter(
    unit_select
).properties(
    title={
      "text": ["Example (YOUR TITLE HERE) "], 
      "subtitle": ["You are viewing XX% of responses from the Kenya Multi-Tiered Framework Survey.", "Drag and drop a box over the scatterplot to filter the histogram, or use the dropdown filters below.", ""],
      "color": "black",
      "subtitleColor": "black"
    }
)

final

In [ ]:
# Save as an HTML file
#final.save('final.html')

# Save as a JSON file
final.save('final.json')